In [ ]:
import torch
import numpy as np

# Flatten inputs: (C, H, W) -> (H*W, C)
C, H, W = input_tensor.shape
X = input_tensor.permute(1, 2, 0).reshape(-1, C).numpy()   # shape: (H*W, 17)

# Flatten labels: choose one-hot -> class index
y = torch.argmax(label_tensor, dim=0).reshape(-1).numpy()  # shape: (H*W,)

patch_size = 16  # or 32, 64
C, H, W = input_tensor.shape
input_patches = input_tensor.unfold(1, patch_size, patch_size).unfold(2, patch_size, patch_size)
# shape = (C, num_patches_H, num_patches_W, patch_size, patch_size)

num_patches_H, num_patches_W = input_patches.shape[1:3]
input_patches = input_patches.permute(1, 2, 0, 3, 4).contiguous().view(-1, C*patch_size*patch_size)
# shape = (num_patches_total, C*patch_size*patch_size)

label_patches = label_tensor.unfold(1, patch_size, patch_size).unfold(2, patch_size, patch_size)
label_patches = label_patches.permute(1, 2, 0, 3, 4).contiguous().view(-1, 4, patch_size*patch_size)
label_patches = torch.argmax(label_patches, dim=1).mode(dim=1)[0].numpy()
# label_patches shape = (num_patches_total,) → most frequent class per patch


import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost
clf = xgb.XGBClassifier(
    max_depth=6,
    n_estimators=100,
    learning_rate=0.1,
    objective='multi:softmax',
    num_class=4
)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
